In [ ]:
# Normalise the intensity of a z-stack, save single channel file for smFISH
# 10 December 2018
# 
#  Select the appropriate channel
#  Measure average intensity of each image
#  Save measurements as an array
#  Calculate stack mean from the array,
#  calculate the difference between the mean intensity of each frame and the stack mean,
#  add the difference from each frame.
#  To normalise an entire dataset, run the script and note the dataset mean intensity from the last file,
#  then replace stackMean in line 74 with the dataset mean intensity value
# 
#  Call from the command line with the following script:
# fiji --headless --console -macro ~/src/FIJI_macros/zstack_normalisation_v7_cmndln.ijm /path/to/directory

import pandas as pd
import numpy as np
import argparse
import sys
import os
from skimage import io

indir="/Users/joshtitlow/tmp/smFISH_data/"
# create a list to store intensity values
dataset_ave_intensity = []

def main(argList):
    
    #parser= argparse.ArgumentParser()
    
    # specify the FQ centroid files and co-detection distance threshold in nm
    #parser.add_argument('-indir', help='FQ_spots file for reference channel')
    #parser.add_argument('-outdir', help='FQ_spots file for co-detection')
    #parser.add_argument('-threshold', help='distance to calculate nearest neighbor')
        
    #args= parser.parse_args(args=argList)

    #infiles = os.listdir(args.indir)
    infiles = os.listdir(indir)
    
    # create list to store data
    filename = []
    co_detect = []
    ch1_spots = []
    ch2_spots = []
    
    # loop through files
    for i in infiles:
        print "processing", i
        
        # open the image using sci kit image
        im = io.imread(indir+i)
        
        # get centroid coordinates
        xpos_ref = ref_file['Pos_X']
        ypos_ref = ref_file['Pos_Y']
        zpos_ref = ref_file['Pos_Z']

        xpos_targ = targ_file['Pos_X']
        ypos_targ = targ_file['Pos_Y']
        zpos_targ = targ_file['Pos_Z']

        # convert data into a numpy array
        target_df = np.column_stack((xpos_targ,ypos_targ,zpos_targ))
        #target_df = [target_df.Pos_Y.str.contains("SPOTS_END") == False]

        # create list to store amplitude of co-detected spot, ref/targ ratio, and target distance
        targ_amp = []
        targ_dist = []

        for index, row in ref_file.iterrows():

            # get 3D position from X,Y,Z position columns
            pt  = row['Pos_X'], row['Pos_Y'], row['Pos_Z']
            print pt
            # find nearest neighbor and calculate distance
            distance,index = spatial.KDTree(target_df).query(pt)

            # add nearest neighbor amp and dist to a list
            targ_amp.append(targ_file['AMP'].iloc[index])
            targ_dist.append(distance)

        # add lists to ref_file
        ref_file['target_amp'] = targ_amp
        ref_file['r_t_ratio'] = ref_file['AMP'].div(targ_amp)
        ref_file['targ_dist'] = targ_dist

        # calculate co-detection percentage and add it to list
        codetect = 100 * (float(len(ref_file[ref_file.targ_dist <                       
            float(args.threshold)])))/(float(len(ref_file.index)))
        co_detect.append(codetect)

        # calculate number of spots and add to list, with filename
        ch1_spots.append(len(ref_file))
        ch2_spots.append(len(targ_file))
        filename.append(i)

        # write ref_file to csv
        #ref_file.to_csv('test_list.csv', index=False)
    
    # add data to dataframe and save
    df = pd.DataFrame({'filename':filename,'co_detect%':co_detect, 'ch1_spots':ch1_spots, 'ch2_spots':ch2_spots})
    df = df[['filename', 'co_detect%', 'ch1_spots', 'ch2_spots']]
    df.to_csv('codetection_stats.csv', index=False)
    
if __name__=='__main__':
    main(sys.argv[1:])

In [73]:
import pandas as pd
import numpy as np
import argparse
import sys
import os
from skimage import io
from skimage import data

indir="/Users/joshtitlow/tmp/smFISH_data/"
channel=1
# create a list to store intensity values
dataset_ave_intensity = []

def main(argList):
    
    #parser= argparse.ArgumentParser()
    
    # specify the FQ centroid files and co-detection distance threshold in nm
    #parser.add_argument('-indir', help='FQ_spots file for reference channel')
    #parser.add_argument('-outdir', help='FQ_spots file for co-detection')
    #parser.add_argument('-channel', help='distance to calculate nearest neighbor')
        
    #args= parser.parse_args(args=argList)

    #infiles = os.listdir(args.indir)
    infiles = os.listdir(indir)
    
    # create list to store data
    filename = []
    co_detect = []
    ch1_spots = []
    ch2_spots = []
    
    # loop through files
    for image in infiles:
        print "processing", image
        im = io.imread(indir+image)
        print im.shape
        print np.mean(im[0])
        #for x in np.nditer(im):
        #    print x
        #zstack_ave=np.mean(im[0])
        #dataset_ave_intensity.append(zstack_ave)

    print "dataset_ave_intensity =", np.mean(dataset_ave_intensity)
if __name__=='__main__':
    main(sys.argv[1:])

SyntaxError: invalid syntax (<ipython-input-73-f2f7541ee883>, line 35)

In [46]:
a = np.arange(6).reshape(2,3)
print a
for x in np.nditer(a):
    print x

[[0 1 2]
 [3 4 5]]
0
1
2
3
4
5


In [68]:
%pylab inline
import cv2
from cv2_rolling_ball import subtract_background_rolling_ball
img = cv2.imread('/Users/joshtitlow/tmp/smFISH_data/20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tiff', 0)
img, background = subtract_background_rolling_ball(img, 30)
#img = plt.imread(img)
plt.imshow(img)

Populating the interactive namespace from numpy and matplotlib


TypeError: can't multiply sequence by non-int of type 'float'

In [62]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/Users/joshtitlow/tmp/smFISH_data/20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tiff)
imgplot = plt.imshow(img)
plt.show()

import matplotlib.pyplot as plt
img = plt.imread('/Users/joshtitlow/tmp/smFISH_data/20171013_CS_2colorsmFISH_msp670_570_DAPI_HRP405_p1s3r.ome.tiff')
plt.imshow(img)
plt.show()

TypeError: Invalid dimensions for image data

In [53]:
img, background = subtract_background_rolling_ball(img, 30, light_background=True, use_paraboloid=False, do_presmooth=True)

TypeError: can't multiply sequence by non-int of type 'float'